In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import warnings
from sklearn.utils import shuffle
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

In [ ]:
def create_class_mapping(csv_file):
    df = pd.read_csv(csv_file)
    classes = df['class'].unique()
    return {name: idx for idx, name in enumerate(classes)}

train_csv = '/content/drive/My Drive/Datasets/Thermal_Images/train_annotations.csv'
class_to_id = create_class_mapping(train_csv)
print("Class to ID mapping:", class_to_id)


Class to ID mapping: {'person': 0, 'dog': 1}


In [ ]:
def process_annotations(csv_file, image_dir, label_dir, class_to_id, img_width, img_height):
    df = pd.read_csv(csv_file)
    df = df[df['class'].notna() & df['class'].isin(class_to_id.keys())]
    os.makedirs(label_dir, exist_ok=True)

    for _, row in df.iterrows():
        file_name = row['filename']
        class_name = row['class']
        if class_name not in class_to_id:
            print(f"Skipping unrecognized class: {class_name} in file {file_name}")
            continue

        class_id = class_to_id[class_name]
        xmin, ymin, xmax, ymax = map(float, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        bbox_width = (xmax - xmin) / img_width
        bbox_height = (ymax - ymin) / img_height
        label_str = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n"

        txt_file_path = os.path.join(label_dir, os.path.splitext(file_name)[0] + '.txt')
        with open(txt_file_path, 'a') as file:
            file.write(label_str)

In [ ]:
def display_sample_images(csv_file, image_dir, num_samples=5):
    df = pd.read_csv(csv_file)
    sample_df = df.sample(n=num_samples)
    plt.figure(figsize=(12, 10))

    for index, (i, row) in enumerate(sample_df.iterrows()):
        img_path = os.path.join(image_dir, row['filename'])
        image = cv2.imread(img_path)
        if image is None:
            print(f"Failed to load image: {img_path}")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

        plt.subplot(num_samples, 1, index + 1)
        plt.imshow(image)
        plt.gca().add_patch(Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='red', facecolor='none'))
        plt.title(f"Class: {row['class']}")

    plt.tight_layout()
    plt.show()



In [ ]:
img_width, img_height = 416, 416


datasets = {
    "train": ('train_annotations.csv', 'train', 'train_labels'),
    "test": ('test_annotations.csv', 'test', 'test_labels'),
    "valid": ('valid_annotations.csv', 'valid', 'valid_labels')
}

for dataset_type, paths in datasets.items():
    csv_file, images_dir, labels_dir = [os.path.join('/content/drive/My Drive/Datasets/Thermal_Images', path) for path in paths]
    process_annotations(csv_file, images_dir, labels_dir, class_to_id, img_width, img_height)


display_sample_images(train_csv, '/content/drive/My Drive/Datasets/Thermal_Images/train')

In [ ]:
!pip install ultralytics

In [ ]:
!pip install pyyaml


In [ ]:
import yaml

data = {
    'path': '/content/drive/My Drive/Datasets/Thermal_Images',
    'train': 'train',
    'val': 'valid',
    'test': 'test',


    'train_labels': 'train_labels',
    'val_labels': 'valid_labels',
    'test_labels': 'test_labels',

    'nc': 2,
    'names': ['Dog', 'Human']
}

yaml_path = '/content/drive/My Drive/Datasets/Thermal_Images/dataset.yaml'

with open(yaml_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)



In [ ]:
from ultralytics import YOLO
import PIL
from PIL import Image
from IPython.display import display
import os
import pathlib
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
import os

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
from ultralytics import YOLO  # Make sure this import works with your setup

def train_yolo_model():
    # Initialize the model (ensure the model weights path and YOLO version are correct)
    model = YOLO('yolov8m.pt')  # Adjust the path to pre-trained weights if needed

    # Start training the model
    model.train(
        data=yaml_path,  # Use the YAML file path created above
        epochs=50,       # Define the number of training epochs
        batch=16,        # Define the batch size
        imgsz=416,       # Standard image size for YOLO is usually 416x416 or 640x640
        device='0',      # GPU device (CUDA) if available, otherwise 'cpu'
        cache=True       # Caching images to RAM for faster training if RAM allows
    )

# Call the function to start training
train_yolo_model()


Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/My Drive/Datasets/Thermal_Images/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /content/drive/My Drive/Datasets/Thermal_Images/train.cache... 0 images, 142 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/My Drive/Datasets/Thermal_Images/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



train: Caching images (0.1GB RAM): 100%|██████████| 142/142 [00:00<00:00, 358.45it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

       3/50      4.12G          0      40.52          0          0        416: 100%|██████████| 9/9 [00:01<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.53it/s]

                   all         41          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50         4G          0      23.74          0          0        416:  11%|█         | 1/9 [00:00<00:01,  5.14it/s]Exception in thread Thread-335 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 53, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 30, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 495, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing

KeyboardInterrupt: 

In [ ]:
import os

def verify_labels(base_path, image_folder, label_folder):
    label_dir = os.path.join(base_path, label_folder)
    image_dir = os.path.join(base_path, image_folder)

    labels = os.listdir(label_dir)
    images = os.listdir(image_dir)

    print(f"Total labels: {len(labels)}, Total images: {len(images)}")

    # Check for matching image files for label files
    for label_file in labels[:5]:  # Check the first 5 label files
        image_file = label_file.replace('.txt', '.jpg')  # Adjust the extension if your images are in a different format
        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, label_file)

        if os.path.exists(image_path):
            print(f"Matching image for {label_file} exists.")
        else:
            print(f"No matching image for {label_file}.")

        with open(label_path, 'r') as file:
            lines = file.readlines()
            if not lines:
                print(f"{label_file} is empty.")
            else:
                print(f"{label_file} contains: {lines}")

base_path = '/content/drive/My Drive/Datasets/Thermal_Images'
verify_labels(base_path, 'train', 'train_labels')  # Adjust as needed for 'valid'/'valid_labels' or 'test'/'test_labels'


Total labels: 131, Total images: 142
Matching image for IMG_0112_jpg.rf.016d04c2af3bc0221a5153d6af8b9f30.txt exists.
IMG_0112_jpg.rf.016d04c2af3bc0221a5153d6af8b9f30.txt contains: ['0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n']
Matching image for IMG_0054_jpg.rf.03e0fd11bad6afeb085f2f156d7fd043.txt exists.
IMG_0054_jpg.rf.03e0fd11bad6afeb085f2f156d7fd043.txt contains: ['1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n']
Matching image for IMG_0094_jpg.rf.03dbd175

In [ ]:
import os

def find_missing_labels(image_dir, label_dir):
    # List all images and label files
    images = {file.split('.')[0] for file in os.listdir(image_dir) if file.endswith('.jpg')}  # Adjust if different image format
    labels = {file.split('.')[0] for file in os.listdir(label_dir) if file.endswith('.txt')}

    # Find images without corresponding label files
    missing_labels = images - labels
    print(f"Images without labels: {len(missing_labels)}")
    for img in missing_labels:
        print(img)

base_path = '/content/drive/My Drive/Datasets/Thermal_Images'
image_dir = os.path.join(base_path, 'train')
label_dir = os.path.join(base_path, 'train_labels')
find_missing_labels(image_dir, label_dir)


Images without labels: 11
IMG_0001_jpg
IMG_0019 3_jpg
IMG_0001 4_jpg
IMG_0008 3_jpg
IMG_0021 3_jpg
IMG_0020 3_jpg
IMG_0005_jpg
IMG_0007 4_jpg
IMG_0055 2_jpg
IMG_0004_jpg
IMG_0054 2_jpg


In [ ]:
def create_empty_labels(image_dir, label_dir):
    images = {file.split('.')[0]: file for file in os.listdir(image_dir) if file.endswith('.jpg')}  # Adjust if different image format
    labels = {file.split('.')[0] for file in os.listdir(label_dir) if file.endswith('.txt')}

    missing_labels = set(images.keys()) - labels
    for img in missing_labels:
        empty_label_path = os.path.join(label_dir, f"{img}.txt")
        open(empty_label_path, 'a').close()  # Creates an empty txt file
        print(f"Created empty label for {img}")

create_empty_labels(image_dir, label_dir)


Created empty label for IMG_0001_jpg
Created empty label for IMG_0019 3_jpg
Created empty label for IMG_0001 4_jpg
Created empty label for IMG_0008 3_jpg
Created empty label for IMG_0021 3_jpg
Created empty label for IMG_0020 3_jpg
Created empty label for IMG_0005_jpg
Created empty label for IMG_0007 4_jpg
Created empty label for IMG_0055 2_jpg
Created empty label for IMG_0004_jpg
Created empty label for IMG_0054 2_jpg


In [ ]:
import os

train_labels_path = '/content/drive/My Drive/Datasets/Thermal_Images/train_labels/'
files = os.listdir(train_labels_path)

# Check the first 5 label files
for file in files[:5]:
    full_path = os.path.join(train_labels_path, file)
    with open(full_path, 'r') as f:
        lines = f.readlines()
        print(f"Contents of {file}: {lines}")  # Shows the content of label files
        if not lines:
            print(f"Warning: {file} is empty.")


Contents of IMG_0112_jpg.rf.016d04c2af3bc0221a5153d6af8b9f30.txt: ['0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n', '0 0.7127403846153846 0.6213942307692307 0.04567307692307692 0.06490384615384616\n']
Contents of IMG_0054_jpg.rf.03e0fd11bad6afeb085f2f156d7fd043.txt: ['1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n', '1 0.4338942307692308 0.546875 0.7139423076923077 0.8677884615384616\n']
Contents of IMG_0094_jpg.rf.03dbd175cdbd5379608debcc783a5361.txt: ['0 0.6454326923076923 0.5444711538461539 0.06490384615384616 0.0985576923076923\n', '0 0.5949519230769231 0.5504807692307693 0.05048076923076923 0.09615384615384616\n'

In [ ]:
# Function to display detection results
def display_detections(model, image_dir):

    images = [img for img in os.listdir(image_dir) if img.endswith('.jpg')]
    random_image_name = random.choice(images)
    image_path = os.path.join(image_dir, random_image_name)
    image = Image.open(image_path)

    # Display the image
    results = model.predict(source=image_path, save=False, conf=0.25, iou=0.45)

    print(results)  # Debug: print results to see the structure

    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    ax = plt.gca()

    # Assuming results.pandas().xyxy[0] works for accessing bounding boxes
    if hasattr(results, 'pandas'):
        bbox_data = results.pandas().xyxy[0]  # Using .pandas() if applicable
    else:
        bbox_data = results  # Fallback if no .pandas() method

    # Draw bounding boxes and labels
    for index, row in bbox_data.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1, f'{row["name"]} {row["confidence"]:.2f}', color='white', fontsize=8, backgroundcolor='red')
    plt.show()

# Example usage: display a random image from the training dataset
display_detections(model, train_images_dir)


image 1/1 /content/drive/My Drive/Datasets/PlantDoc Dataset/train/22+August+2013+010-1_jpg.rf.63a6322b8174b0ae85c42d43a9603406.jpg: 640x640 (no detections), 37.7ms
Speed: 2.8ms preprocess, 37.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 4

AttributeError: 'list' object has no attribute 'iterrows'